In [1]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

num_var = 10
num_ineq = 10
num_eq = 10
num_examples = 10000

np.random.seed(17)
Q = np.diag(np.random.random(num_var))
p = np.random.random(num_var)
A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
X = np.random.uniform(-1, 1, size=(num_examples, num_eq))
G = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
h = np.sum(np.abs(G@np.linalg.pinv(A)), axis=1)

QP_problem = SimpleProblem(Q, p, A, G, h, X)
QP_problem.calc_Y()
print(len(QP_problem.Y))

running osqp
10000


In [2]:
import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict

from primal_dual_refactored import PrimalDualTrainer
from gep_primal_dual_main_refactored import prep_data
from gep_problem_refactored import GEPProblemSet
from gep_main import run_model as run_Gurobi
from gep_main import run_model_no_bounds as run_Gurobi_no_bounds
from get_gurobi_vars import OptValueExtractor

import pyomo as pyo

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"


SCALE_FACTORS = {
    "pDemand": 1/1000,  # MW -> GW
    "pGenAva": 1,       # Don't scale
    "pVOLL": 1,         # kEUR/MWh -> mEUR/GWh
    "pWeight": 1,       # Don't scale
    "pRamping": 1,      # Don't scale
    "pInvCost": 1,      # kEUR/MW -> mEUR/GW
    "pVarCost": 1,      # kEUR/MWh -> mEUR/GWh
    "pUnitCap": 1/1000, # MW -> GW
    "pExpCap": 1/1000,  # MW -> GW
    "pImpCap": 1/1000,  # MW -> GW
}


## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

for i, experiment_instance in enumerate(experiment["experiments"]):
    # Prep proble data:
    gep_data = prep_data(experiment_instance, sample_duration=12)


Running on cpu
Parsing the config file
Initializing the solver
Using Gurobi
Parsing the config file
Parsing the config file
Wrangling the input data
Creating problem instance
Populating ineq constraints
Populating eq constraints
Creating objective coefficients
Creating input for NN: X
Size of train set: 584
Size of val set: 73
Size of test set: 73


In [24]:
gep_X = gep_data.X[gep_data.train_indices]
gep_eq_cm = gep_data.eq_cm
gep_ineq_cm = gep_data.ineq_cm
gep_eq_rhs = gep_data.eq_rhs
gep_ineq_rhs = gep_data.ineq_rhs

opt_targets = gep_data.opt_targets
opt_decision_variables = opt_targets[0]["y"]
print(len(opt_decision_variables))

p_gt_idx = len(gep_data.G)*gep_data.sample_duration
f_lt_idx = p_gt_idx + len(gep_data.L)*gep_data.sample_duration
md_nt_idx = f_lt_idx + len(gep_data.N)*gep_data.sample_duration
ui_g_idx = md_nt_idx + len(gep_data.G)
p_gt = opt_decision_variables[:p_gt_idx]
f_lt = opt_decision_variables[p_gt_idx:f_lt_idx]
md_nt = opt_decision_variables[f_lt_idx:md_nt_idx]
ui_g = opt_decision_variables[md_nt_idx:]
print(p_gt)
print(f_lt)
print(md_nt)
print(ui_g)

111
tensor([    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
            0.0000,     0.0000,  3986.2258,     0.0000,     0.0000,     0.0000,
            0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
            0.0000,     0.0000, 33287.9487, 24001.4938, 27050.9301, 31825.7683,
            0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
            0.0000,     0.0000,  5067.1065, 21213.1374, 21749.9797, 20969.2613])
tensor([-2000.0000, -2000.0000, -2000.0000, -2000.0000, -2000.0000, -2000.0000,
        -2000.0000, -2000.0000, -2000.0000,  -275.3995,  -502.7433, -2000.0000,
         2000.0000,  2000.0000,  2000.0000,  2000.0000,  2000.0000,  2000.0000,
         2000.0000,  2000.0000,  2400.0000, -3400.0000, -3400.0000, -2151.6456,
        -2000.0000, -2000.0000, -2000.0000, -2000.0000, -2000.0000, -2000.0000,
        -2000.0000, -2000.0000,  5000.0000, -5000.0000, -5000.0000, -5000.0000])
tensor([ 5336.3813,  4884.6334,  4